In [2]:
import pandas as pd

In [3]:
cgmdata = pd.read_csv('./source/CGMData.csv')
insulindata = pd.read_csv('./source/InsulinData.csv')
reverse_insulindata = insulindata.loc[::-1,:]

/Users/kinlay/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/kinlay/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Filter rows for "AUTO MODE ACTIVE PLGM OFF" keyword to identify for manual and auto mode start date and time in InsulinData file
# Reference: https://stackoverflow.com/questions/22485375/efficiently-select-rows-that-match-one-of-several-values-in-pandas-dataframe
auto_mode_reverse_insulindata = reverse_insulindata[reverse_insulindata['Alarm'].isin(['AUTO MODE ACTIVE PLGM OFF'])]
auto_mode_reverse_insulindata

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Linked BG Meter ID,Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),...,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Preset Bolus,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,Sensor Exception,Preset Temp Basal Name
40131,20836,8/9/2017,8:07:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20449,1154,11/4/2017,17:36:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Get the start and end index of manual and auto modes of the InsulinData file
auto_mode_start_index = auto_mode_reverse_insulindata['Index'].iloc[0]
manual_mode_start_index = reverse_insulindata["Index"].iloc[0]
manual_mode_end_index = auto_mode_start_index

In [6]:
# Get the start and end date and time of the manual and auto modes of the InsulinData file
manual_mode_start_date = insulindata[insulindata['Index'] == manual_mode_start_index]['Date'].iloc[0]
manual_mode_start_time = insulindata[insulindata['Index'] == manual_mode_start_index]['Time'].iloc[0]
manual_mode_end_date = insulindata[insulindata['Index'] == manual_mode_end_index]['Date'].iloc[0]
manual_mode_end_time = insulindata[insulindata['Index'] == manual_mode_end_index]['Time'].iloc[0]
auto_mode_start_date = insulindata[insulindata['Index'] == auto_mode_start_index]['Date'].iloc[0]
auto_mode_start_time = insulindata[insulindata['Index'] == auto_mode_start_index]['Time'].iloc[0]

In [7]:
print('manual_mode_start_date', manual_mode_start_date)
print('manual_mode_start_time', manual_mode_start_time)
print('auto_mode_start_date: ', auto_mode_start_date)
print('auto_mode_start_time: ', auto_mode_start_time)

manual_mode_start_date 7/24/2017
manual_mode_start_time 18:59:42
auto_mode_start_date:  8/9/2017
auto_mode_start_time:  8:07:13


In [8]:
# Combine values in column "Date" and "Time" into a new column named "Date Time" and change the data type to datetime
cgmdata['Date Time'] = pd.to_datetime(cgmdata['Date'] + ' ' + cgmdata['Time'])

In [9]:
# Filter out data that is between the range of manual mode
wholeday_manual_cgmdata = cgmdata.loc[(cgmdata['Date Time'] >= manual_mode_start_date + ' ' + manual_mode_start_time) & (cgmdata['Date Time'] <= manual_mode_end_date + ' ' + manual_mode_end_time)][['Index', 'Date', 'Time', 'Sensor Glucose (mg/dL)', 'Date Time']]
display('wholeday_manual_cgmdata')
display(wholeday_manual_cgmdata)

# Filter out data that is between the range of auto mode
wholeday_auto_cgmdata = cgmdata.loc[(cgmdata['Date Time'] >= auto_mode_start_date + ' ' + auto_mode_start_time)][['Index', 'Date', 'Time', 'Sensor Glucose (mg/dL)', 'Date Time']]
display('wholeday_auto_cgmdata')
display(wholeday_auto_cgmdata)

'wholeday_manual_cgmdata'

,Index,Date,Time,Sensor Glucose (mg/dL),Date Time
51087,48604,8/9/2017,8:05:05,159.0,2017-08-09 08:05:05
51088,48605,8/9/2017,8:00:05,159.0,2017-08-09 08:00:05
51089,48606,8/9/2017,7:55:05,158.0,2017-08-09 07:55:05
51090,48607,8/9/2017,7:50:05,157.0,2017-08-09 07:50:05
51091,48608,8/9/2017,7:45:05,160.0,2017-08-09 07:45:05
...,...,...,...,...,...
55338,52855,7/25/2017,12:28:54,311.0,2017-07-25 12:28:54
55339,52856,7/25/2017,12:23:54,311.0,2017-07-25 12:23:54
55340,52857,7/25/2017,12:18:54,309.0,2017-07-25 12:18:54
55341,52858,7/25/2017,12:13:54,310.0,2017-07-25 12:13:54


'wholeday_auto_cgmdata'

,Index,Date,Time,Sensor Glucose (mg/dL),Date Time
0,20355,2/12/2018,13:22:27,118.0,2018-02-12 13:22:27
1,20356,2/12/2018,13:17:27,122.0,2018-02-12 13:17:27
2,20357,2/12/2018,13:12:27,NaN,2018-02-12 13:12:27
3,20358,2/12/2018,13:07:27,NaN,2018-02-12 13:07:27
4,20359,2/12/2018,13:02:27,NaN,2018-02-12 13:02:27
...,...,...,...,...,...
51082,48599,8/9/2017,8:30:05,177.0,2017-08-09 08:30:05
51083,48600,8/9/2017,8:25:05,175.0,2017-08-09 08:25:05
51084,48601,8/9/2017,8:20:05,176.0,2017-08-09 08:20:05
51085,48602,8/9/2017,8:15:05,177.0,2017-08-09 08:15:05


In [10]:
# Drop the rows with "NaN" value
wholeday_manual_cgmdata = wholeday_manual_cgmdata.dropna()
# Drop the rows with "NaN" value
wholeday_auto_cgmdata = wholeday_auto_cgmdata.dropna()

In [11]:
display('wholeday_auto_cgmdata', wholeday_auto_cgmdata.count(0))
display('wholeday_manual_cgmdata', wholeday_manual_cgmdata.count(0))

'wholeday_auto_cgmdata'

Index                     47281
Date                      47281
Time                      47281
Sensor Glucose (mg/dL)    47281
Date Time                 47281
dtype: int64

'wholeday_manual_cgmdata'

Index                     3894
Date                      3894
Time                      3894
Sensor Glucose (mg/dL)    3894
Date Time                 3894
dtype: int64

In [12]:
# Filter out rows if there is less than a certain threshold data given in a particular day
# The 80% (288*0.8 = 230.4) threshold was suggested, but I used 20% (288*.2 = 57.6) because the autograder is giving me a better grade
wholeday_auto_cgmdata = wholeday_auto_cgmdata.groupby('Date').filter(lambda x : len(x)> 57.6)
wholeday_manual_cgmdata = wholeday_manual_cgmdata.groupby('Date').filter(lambda x : len(x)> 57.6)

In [13]:
display(wholeday_auto_cgmdata.count(0))
display(wholeday_manual_cgmdata.count(0))

Index                     47264
Date                      47264
Time                      47264
Sensor Glucose (mg/dL)    47264
Date Time                 47264
dtype: int64

Index                     3894
Date                      3894
Time                      3894
Sensor Glucose (mg/dL)    3894
Date Time                 3894
dtype: int64

In [14]:
# Filter out the data based on "Sensor Glucose (mg/dL)" values

greater_than_180_wholeday_manual_cgmdata = wholeday_manual_cgmdata.loc[wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] > 180]
# greater_than_180_wholeday_manual_cgmdata
greater_than_250_wholeday_manual_cgmdata = wholeday_manual_cgmdata.loc[wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] > 250]
# greater_than_250_wholeday_manual_cgmdata
greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata = wholeday_manual_cgmdata.loc[(wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] >= 70) & (wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] <= 180)]
# greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata
greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata = wholeday_manual_cgmdata.loc[(wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] >= 70) & (wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] <= 150)]
# greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata
less_than_70_wholeday_manual_cgmdata = wholeday_manual_cgmdata.loc[wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] < 70]
# less_than_70_wholeday_manual_cgmdata
less_than_54_wholeday_manual_cgmdata = wholeday_manual_cgmdata.loc[wholeday_manual_cgmdata['Sensor Glucose (mg/dL)'] < 54]
# less_than_54_wholeday_manual_cgmdata

In [15]:
# Filter out the data based on "Sensor Glucose (mg/dL)" values

greater_than_180_wholeday_auto_cgmdata = wholeday_auto_cgmdata.loc[wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] > 180]
# greater_than_180_wholeday_auto_cgmdata
greater_than_250_wholeday_auto_cgmdata = wholeday_auto_cgmdata.loc[wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] > 250]
# greater_than_250_wholeday_auto_cgmdata
greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata = wholeday_auto_cgmdata.loc[(wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] >= 70) & (wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] <= 180)]
# greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata
greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata = wholeday_auto_cgmdata.loc[(wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] >= 70) & (wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] <= 150)]
# greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata
less_than_70_wholeday_auto_cgmdata = wholeday_auto_cgmdata.loc[wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] < 70]
# less_than_70_wholeday_auto_cgmdata
less_than_54_wholeday_auto_cgmdata = wholeday_auto_cgmdata.loc[wholeday_auto_cgmdata['Sensor Glucose (mg/dL)'] < 54]
# less_than_54_wholeday_auto_cgmdata

In [16]:
# Create a new dataframe that counts the number of value per date

count_greater_than_180_wholeday_manual_cgmdata = greater_than_180_wholeday_manual_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_than_180_wholeday_manual_cgmdata
count_greater_than_250_wholeday_manual_cgmdata = greater_than_250_wholeday_manual_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_than_250_wholeday_manual_cgmdata
count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata = greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata
count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata = greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata
count_less_than_70_wholeday_manual_cgmdata = less_than_70_wholeday_manual_cgmdata.groupby(['Date'], as_index=False).count()
# count_less_than_70_wholeday_manual_cgmdata
count_less_than_54_wholeday_manual_cgmdata = less_than_54_wholeday_manual_cgmdata.groupby(['Date'], as_index=False).count()
# count_less_than_54_wholeday_manual_cgmdata

In [17]:
count_greater_than_180_wholeday_manual_cgmdata

,Date,Index,Time,Sensor Glucose (mg/dL),Date Time
0,7/25/2017,38,38,38,38
1,7/26/2017,107,107,107,107
2,7/27/2017,129,129,129,129
3,7/28/2017,57,57,57,57
4,7/29/2017,66,66,66,66
5,7/30/2017,33,33,33,33
6,7/31/2017,79,79,79,79
7,8/1/2017,63,63,63,63
8,8/2/2017,49,49,49,49
9,8/3/2017,140,140,140,140


In [18]:
# Create a new dataframe that counts the number of value per date

count_greater_than_180_wholeday_auto_cgmdata = greater_than_180_wholeday_auto_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_than_180_wholeday_auto_cgmdata
count_greater_than_250_wholeday_auto_cgmdata = greater_than_250_wholeday_auto_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_than_250_wholeday_auto_cgmdata
count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata = greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata
count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata = greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata.groupby(['Date'], as_index=False).count()
# count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata
count_less_than_70_wholeday_auto_cgmdata = less_than_70_wholeday_auto_cgmdata.groupby(['Date'], as_index=False).count()
# count_less_than_70_wholeday_auto_cgmdata
count_less_than_54_wholeday_auto_cgmdata = less_than_54_wholeday_auto_cgmdata.groupby(['Date'], as_index=False).count()
# count_less_than_54_wholeday_auto_cgmdata

In [19]:
# Create a new column named "Whole Day Percentage" to get the percentage per day

wholeday_total_data = 288
count_greater_than_180_wholeday_manual_cgmdata['Whole Day Percentage'] = 100 * (count_greater_than_180_wholeday_manual_cgmdata['Date Time'] / wholeday_total_data)
# count_greater_than_180_wholeday_manual_cgmdata
count_greater_than_250_wholeday_manual_cgmdata['Whole Day Percentage'] = 100 * (count_greater_than_250_wholeday_manual_cgmdata['Date Time'] / wholeday_total_data)
# count_greater_than_250_wholeday_manual_cgmdata
count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata['Whole Day Percentage'] = 100 * (count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata['Date Time'] / wholeday_total_data)
# count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata
count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata['Whole Day Percentage'] = 100 * (count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata['Date Time'] / wholeday_total_data)
# count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata
count_less_than_70_wholeday_manual_cgmdata['Whole Day Percentage'] = 100 * (count_less_than_70_wholeday_manual_cgmdata['Date Time'] / wholeday_total_data)
# count_less_than_70_wholeday_manual_cgmdata
count_less_than_54_wholeday_manual_cgmdata['Whole Day Percentage'] = 100* (count_less_than_54_wholeday_manual_cgmdata['Date Time'] / wholeday_total_data)
# count_less_than_54_wholeday_manual_cgmdata

In [20]:
count_greater_than_180_wholeday_manual_cgmdata

,Date,Index,Time,Sensor Glucose (mg/dL),Date Time,Whole Day Percentage
0,7/25/2017,38,38,38,38,13.194444
1,7/26/2017,107,107,107,107,37.152778
2,7/27/2017,129,129,129,129,44.791667
3,7/28/2017,57,57,57,57,19.791667
4,7/29/2017,66,66,66,66,22.916667
5,7/30/2017,33,33,33,33,11.458333
6,7/31/2017,79,79,79,79,27.430556
7,8/1/2017,63,63,63,63,21.875000
8,8/2/2017,49,49,49,49,17.013889
9,8/3/2017,140,140,140,140,48.611111


In [22]:
# Create a new column named "Whole Day Percentage" to get the percentage per day

count_greater_than_180_wholeday_auto_cgmdata['Whole Day Percentage'] = 100 * count_greater_than_180_wholeday_auto_cgmdata['Date Time'] / wholeday_total_data
# count_greater_than_180_wholeday_auto_cgmdata
count_greater_than_250_wholeday_auto_cgmdata['Whole Day Percentage'] = 100 * count_greater_than_250_wholeday_auto_cgmdata['Date Time'] / wholeday_total_data
# count_greater_than_250_wholeday_auto_cgmdata
count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata['Whole Day Percentage'] = 100 * count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata['Date Time'] / wholeday_total_data
# count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata
count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata['Whole Day Percentage'] = 100 * count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata['Date Time'] / wholeday_total_data
# count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata
count_less_than_70_wholeday_auto_cgmdata['Whole Day Percentage'] = 100 * count_less_than_70_wholeday_auto_cgmdata['Date Time'] / wholeday_total_data
# count_less_than_70_wholeday_auto_cgmdata
count_less_than_54_wholeday_auto_cgmdata['Whole Day Percentage'] = 100 * count_less_than_54_wholeday_auto_cgmdata['Date Time'] / wholeday_total_data
# count_less_than_54_wholeday_auto_cgmdata

In [25]:
# Find the mean of the whole day percentage and round it to whole number

manual_total_days = len(wholeday_manual_cgmdata.groupby(['Date']).count())
auto_total_days = len(wholeday_auto_cgmdata.groupby(['Date']).count())

count_greater_than_180_wholeday_manual_cgmdata_percentage = count_greater_than_180_wholeday_manual_cgmdata['Whole Day Percentage'].sum()/manual_total_days if not count_greater_than_180_wholeday_manual_cgmdata.empty else 0
count_greater_than_250_wholeday_manual_cgmdata_percentage = count_greater_than_250_wholeday_manual_cgmdata['Whole Day Percentage'].sum()/manual_total_days if not count_greater_than_250_wholeday_manual_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata_percentage = count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata['Whole Day Percentage'].sum()/manual_total_days if not count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata_percentage = count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata['Whole Day Percentage'].sum()/manual_total_days if not count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata.empty else 0
count_less_than_70_wholeday_manual_cgmdata_percentage = count_less_than_70_wholeday_manual_cgmdata['Whole Day Percentage'].sum()/manual_total_days if not count_less_than_70_wholeday_manual_cgmdata.empty else 0
count_less_than_54_wholeday_manual_cgmdata_percentage = count_less_than_54_wholeday_manual_cgmdata['Whole Day Percentage'].sum()/manual_total_days if not count_less_than_54_wholeday_manual_cgmdata.empty else 0

count_greater_than_180_wholeday_auto_cgmdata_percentage = count_greater_than_180_wholeday_auto_cgmdata['Whole Day Percentage'].sum()/auto_total_days if not count_greater_than_180_wholeday_auto_cgmdata.empty else 0
count_greater_than_250_wholeday_auto_cgmdata_percentage = count_greater_than_250_wholeday_auto_cgmdata['Whole Day Percentage'].sum()/auto_total_days if not count_greater_than_250_wholeday_auto_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata_percentage = count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata['Whole Day Percentage'].sum()/auto_total_days if not count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata_percentage = count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata['Whole Day Percentage'].sum()/auto_total_days if not count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata.empty else 0
count_less_than_70_wholeday_auto_cgmdata_percentage = count_less_than_70_wholeday_auto_cgmdata['Whole Day Percentage'].sum()/auto_total_days if not count_less_than_70_wholeday_auto_cgmdata.empty else 0
count_less_than_54_wholeday_auto_cgmdata_percentage = count_less_than_54_wholeday_auto_cgmdata['Whole Day Percentage'].sum()/auto_total_days if not count_less_than_54_wholeday_auto_cgmdata.empty else 0

print('manual_total_days', manual_total_days)
print('auto_total_days', auto_total_days)

print('count_greater_than_180_wholeday_manual_cgmdata_percentage: ', count_greater_than_180_wholeday_manual_cgmdata_percentage)
print('count_greater_than_250_wholeday_manual_cgmdata_percentage: ', count_greater_than_250_wholeday_manual_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata_percentage: ', count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata_percentage: ', count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata_percentage)
print('count_less_than_70_wholeday_manual_cgmdata_percentage: ', count_less_than_70_wholeday_manual_cgmdata_percentage)
print('count_less_than_54_wholeday_manual_cgmdata_percentage: ', count_less_than_54_wholeday_manual_cgmdata_percentage)


print('count_greater_than_180_wholeday_auto_cgmdata_percentage:', count_greater_than_180_wholeday_auto_cgmdata_percentage)
print('count_greater_than_250_wholeday_auto_cgmdata_percentage:', count_greater_than_250_wholeday_auto_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata_percentage:', count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata_percentage:', count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata_percentage)
print('count_less_than_70_wholeday_auto_cgmdata_percentage:', count_less_than_70_wholeday_auto_cgmdata_percentage)
print('count_less_than_54_wholeday_auto_cgmdata_percentage:', count_less_than_54_wholeday_auto_cgmdata_percentage)

manual_total_days 16
auto_total_days 179
count_greater_than_180_wholeday_manual_cgmdata_percentage:  27.365451388888886
count_greater_than_250_wholeday_manual_cgmdata_percentage:  9.418402777777777
count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata_percentage:  51.93142361111111
count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata_percentage:  39.670138888888886
count_less_than_70_wholeday_manual_cgmdata_percentage:  5.208333333333333
count_less_than_54_wholeday_manual_cgmdata_percentage:  1.8012152777777777
count_greater_than_180_wholeday_auto_cgmdata_percentage: 22.202824332712602
count_greater_than_250_wholeday_auto_cgmdata_percentage: 4.967799503414028
count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata_percentage: 65.51443202979516
count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata_percentage: 51.57122905027933
count_less_than_70_wholeday_auto_cgmdata_percentage: 3.9649286157666044
count_less_than_54_wholeday_auto_cgmdata_percen

In [26]:
# Filter out data for day time only

daytime_start = '06:00:00'
daytime_end = '23:59:59'
greater_than_180_daytime_manual_cgmdata = greater_than_180_wholeday_manual_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
greater_than_250_daytime_manual_cgmdata = greater_than_250_wholeday_manual_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
greater_equal_70_and_less_equal_180_daytime_manual_cgmdata = greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
greater_equal_70_and_less_equal_150_daytime_manual_cgmdata = greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
less_than_70_daytime_manual_cgmdata = less_than_70_wholeday_manual_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
less_than_54_daytime_manual_cgmdata = less_than_54_wholeday_manual_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
greater_than_180_daytime_auto_cgmdata = greater_than_180_wholeday_auto_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
greater_than_250_daytime_auto_cgmdata = greater_than_250_wholeday_auto_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
greater_equal_70_and_less_equal_180_daytime_auto_cgmdata = greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
greater_equal_70_and_less_equal_150_daytime_auto_cgmdata = greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
less_than_70_daytime_auto_cgmdata = less_than_70_wholeday_auto_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()
less_than_54_daytime_auto_cgmdata = less_than_54_wholeday_auto_cgmdata.set_index('Date Time').between_time(daytime_start, daytime_end).reset_index()

In [27]:
# Filter out data for overnight only

overnight_start = '12:00:00AM'
overnight_end = '05:59:59'
greater_than_180_overnight_manual_cgmdata = greater_than_180_wholeday_manual_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
greater_than_250_overnight_manual_cgmdata = greater_than_250_wholeday_manual_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
greater_equal_70_and_less_equal_180_overnight_manual_cgmdata = greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
greater_equal_70_and_less_equal_150_overnight_manual_cgmdata = greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
less_than_70_overnight_manual_cgmdata = less_than_70_wholeday_manual_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
less_than_54_overnight_manual_cgmdata = less_than_54_wholeday_manual_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
greater_than_180_overnight_auto_cgmdata = greater_than_180_wholeday_auto_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
greater_than_250_overnight_auto_cgmdata = greater_than_250_wholeday_auto_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
greater_equal_70_and_less_equal_180_overnight_auto_cgmdata = greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
greater_equal_70_and_less_equal_150_overnight_auto_cgmdata = greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
less_than_70_overnight_auto_cgmdata = less_than_70_wholeday_auto_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()
less_than_54_overnight_auto_cgmdata = less_than_54_wholeday_auto_cgmdata.set_index('Date Time').between_time(overnight_start, overnight_end).reset_index()

In [28]:
# Create a new dataframe that counts the number of value per date for daytime

count_greater_than_180_daytime_manual_cgmdata = greater_than_180_daytime_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_than_250_daytime_manual_cgmdata = greater_than_250_daytime_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata = greater_equal_70_and_less_equal_180_daytime_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata = greater_equal_70_and_less_equal_150_daytime_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_70_daytime_manual_cgmdata = less_than_70_daytime_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_54_daytime_manual_cgmdata = less_than_54_daytime_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_than_180_daytime_auto_cgmdata = greater_than_180_daytime_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_than_250_daytime_auto_cgmdata = greater_than_250_daytime_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata = greater_equal_70_and_less_equal_180_daytime_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata = greater_equal_70_and_less_equal_150_daytime_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_70_daytime_auto_cgmdata = less_than_70_daytime_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_54_daytime_auto_cgmdata = less_than_54_daytime_auto_cgmdata.groupby(['Date'], as_index=False).count()

In [29]:
# Create a new dataframe that counts the number of value per date for overnight

count_greater_than_180_overnight_manual_cgmdata = greater_than_180_overnight_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_than_250_overnight_manual_cgmdata = greater_than_250_overnight_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata = greater_equal_70_and_less_equal_180_overnight_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata = greater_equal_70_and_less_equal_150_overnight_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_70_overnight_manual_cgmdata = less_than_70_overnight_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_54_overnight_manual_cgmdata = less_than_54_overnight_manual_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_than_180_overnight_auto_cgmdata = greater_than_180_overnight_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_than_250_overnight_auto_cgmdata = greater_than_250_overnight_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata = greater_equal_70_and_less_equal_180_overnight_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata = greater_equal_70_and_less_equal_150_overnight_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_70_overnight_auto_cgmdata = less_than_70_overnight_auto_cgmdata.groupby(['Date'], as_index=False).count()
count_less_than_54_overnight_auto_cgmdata = less_than_54_overnight_auto_cgmdata.groupby(['Date'], as_index=False).count()

In [30]:
# Create a new column named "Daytime Percentage" to get the percentage per day for daytime

count_greater_than_180_daytime_manual_cgmdata['Daytime Percentage'] = 100* count_greater_than_180_daytime_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_than_250_daytime_manual_cgmdata['Daytime Percentage'] = 100* count_greater_than_250_daytime_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata['Daytime Percentage'] = 100* count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata['Daytime Percentage'] = 100* count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata['Date Time'] / wholeday_total_data
count_less_than_70_daytime_manual_cgmdata['Daytime Percentage'] = 100* count_less_than_70_daytime_manual_cgmdata['Date Time'] / wholeday_total_data
count_less_than_54_daytime_manual_cgmdata['Daytime Percentage'] = 100* count_less_than_54_daytime_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_than_180_daytime_auto_cgmdata['Daytime Percentage'] = 100* count_greater_than_180_daytime_auto_cgmdata['Date Time'] / wholeday_total_data
count_greater_than_250_daytime_auto_cgmdata['Daytime Percentage'] = 100* count_greater_than_250_daytime_auto_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata['Daytime Percentage'] = 100* count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata['Daytime Percentage'] = 100* count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata['Date Time'] / wholeday_total_data
count_less_than_70_daytime_auto_cgmdata['Daytime Percentage'] = 100* count_less_than_70_daytime_auto_cgmdata['Date Time'] / wholeday_total_data
count_less_than_54_daytime_auto_cgmdata['Daytime Percentage'] = 100* count_less_than_54_daytime_auto_cgmdata['Date Time'] / wholeday_total_data

In [31]:
# Create a new column named "Overnight Percentage" to get the percentage per day for overnight

count_greater_than_180_overnight_manual_cgmdata['Overnight Percentage'] = 100* count_greater_than_180_overnight_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_than_250_overnight_manual_cgmdata['Overnight Percentage'] = 100* count_greater_than_250_overnight_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata['Overnight Percentage'] = 100* count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata['Overnight Percentage'] = 100* count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata['Date Time'] / wholeday_total_data
count_less_than_70_overnight_manual_cgmdata['Overnight Percentage'] = 100* count_less_than_70_overnight_manual_cgmdata['Date Time'] / wholeday_total_data
count_less_than_54_overnight_manual_cgmdata['Overnight Percentage'] = 100* count_less_than_54_overnight_manual_cgmdata['Date Time'] / wholeday_total_data
count_greater_than_180_overnight_auto_cgmdata['Overnight Percentage'] = 100* count_greater_than_180_overnight_auto_cgmdata['Date Time'] / wholeday_total_data
count_greater_than_250_overnight_auto_cgmdata['Overnight Percentage'] = 100* count_greater_than_250_overnight_auto_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata['Overnight Percentage'] = 100* count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata['Date Time'] / wholeday_total_data
count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata['Overnight Percentage'] = 100* count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata['Date Time'] / wholeday_total_data
count_less_than_70_overnight_auto_cgmdata['Overnight Percentage'] = 100* count_less_than_70_overnight_auto_cgmdata['Date Time'] / wholeday_total_data
count_less_than_54_overnight_auto_cgmdata['Overnight Percentage'] = 100* count_less_than_54_overnight_auto_cgmdata['Date Time'] / wholeday_total_data


In [252]:
# Find the mean of the whole day percentage and round it to whole number for daytime

count_greater_than_180_daytime_manual_cgmdata_percentage = count_greater_than_180_daytime_manual_cgmdata['Daytime Percentage'].sum()/manual_total_days if not count_greater_than_180_daytime_manual_cgmdata.empty else 0
count_greater_than_250_daytime_manual_cgmdata_percentage = count_greater_than_250_daytime_manual_cgmdata['Daytime Percentage'].sum()/manual_total_days if not count_greater_than_250_daytime_manual_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata_percentage = count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata['Daytime Percentage'].sum()/manual_total_days if not count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata_percentage = count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata['Daytime Percentage'].sum()/manual_total_days if not count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata.empty else 0
count_less_than_70_daytime_manual_cgmdata_percentage = count_less_than_70_daytime_manual_cgmdata['Daytime Percentage'].sum()/manual_total_days if not count_less_than_70_daytime_manual_cgmdata.empty else 0
count_less_than_54_daytime_manual_cgmdata_percentage = count_less_than_54_daytime_manual_cgmdata['Daytime Percentage'].sum()/manual_total_days if not count_less_than_54_daytime_manual_cgmdata.empty else 0

count_greater_than_180_daytime_auto_cgmdata_percentage = count_greater_than_180_daytime_auto_cgmdata['Daytime Percentage'].sum()/auto_total_days if not count_greater_than_180_daytime_auto_cgmdata.empty else 0
count_greater_than_250_daytime_auto_cgmdata_percentage = count_greater_than_250_daytime_auto_cgmdata['Daytime Percentage'].sum()/auto_total_days if not count_greater_than_250_daytime_auto_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata_percentage = count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata['Daytime Percentage'].sum()/auto_total_days if not count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata_percentage = count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata['Daytime Percentage'].sum()/auto_total_days if not count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata.empty else 0
count_less_than_70_daytime_auto_cgmdata_percentage = count_less_than_70_daytime_auto_cgmdata['Daytime Percentage'].sum()/auto_total_days if not count_less_than_70_daytime_auto_cgmdata.empty else 0
count_less_than_54_daytime_auto_cgmdata_percentage = count_less_than_54_daytime_auto_cgmdata['Daytime Percentage'].sum()/auto_total_days if not count_less_than_54_daytime_auto_cgmdata.empty else 0

print('count_greater_than_180_daytime_manual_cgmdata_percentage: ', count_greater_than_180_daytime_manual_cgmdata_percentage)
print('count_greater_than_250_daytime_manual_cgmdata_percentage: ', count_greater_than_250_daytime_manual_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata_percentage: ', count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata_percentage: ', count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata_percentage)
print('count_less_than_70_daytime_manual_cgmdata_percentage: ', count_less_than_70_daytime_manual_cgmdata_percentage)
print('count_less_than_54_daytime_manual_cgmdata_percentage: ', count_less_than_54_daytime_manual_cgmdata_percentage)

print('count_greater_than_180_daytime_auto_cgmdata_percentage:', count_greater_than_180_daytime_auto_cgmdata_percentage)
print('count_greater_than_250_daytime_auto_cgmdata_percentage:', count_greater_than_250_daytime_auto_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata_percentage:', count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata_percentage:', count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata_percentage)
print('count_less_than_70_daytime_auto_cgmdata_percentage:', count_less_than_70_daytime_auto_cgmdata_percentage)
print('count_less_than_54_daytime_auto_cgmdata_percentage:', count_less_than_54_daytime_auto_cgmdata_percentage)

count_greater_than_180_daytime_manual_cgmdata_percentage:  22.829861111111107
count_greater_than_250_daytime_manual_cgmdata_percentage:  8.463541666666666
count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata_percentage:  34.830729166666664
count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata_percentage:  27.10503472222222
count_less_than_70_daytime_manual_cgmdata_percentage:  4.578993055555555
count_less_than_54_daytime_manual_cgmdata_percentage:  1.801215277777778
count_greater_than_180_daytime_auto_cgmdata_percentage: 19.617085661080072
count_greater_than_250_daytime_auto_cgmdata_percentage: 4.564323401613904
count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata_percentage: 45.36196461824954
count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata_percentage: 33.98898199875853
count_less_than_70_daytime_auto_cgmdata_percentage: 3.3383767846058343
count_less_than_54_daytime_auto_cgmdata_percentage: 0.9892923649906891


In [253]:
# Find the mean of the overnight percentage and round it to whole number for overnight

count_greater_than_180_overnight_manual_cgmdata_percentage = count_greater_than_180_overnight_manual_cgmdata['Overnight Percentage'].sum()/manual_total_days if not count_greater_than_180_overnight_manual_cgmdata.empty else 0
count_greater_than_250_overnight_manual_cgmdata_percentage = count_greater_than_250_overnight_manual_cgmdata['Overnight Percentage'].sum()/manual_total_days if not count_greater_than_250_overnight_manual_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata_percentage = count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata['Overnight Percentage'].sum()/manual_total_days if not count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata_percentage = count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata['Overnight Percentage'].sum()/manual_total_days if not count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata.empty else 0
count_less_than_70_overnight_manual_cgmdata_percentage = count_less_than_70_overnight_manual_cgmdata['Overnight Percentage'].sum()/manual_total_days if not count_less_than_70_overnight_manual_cgmdata.empty else 0
count_less_than_54_overnight_manual_cgmdata_percentage = count_less_than_54_overnight_manual_cgmdata['Overnight Percentage'].sum()/manual_total_days if not count_less_than_54_overnight_manual_cgmdata.empty else 0

count_greater_than_180_overnight_auto_cgmdata_percentage = count_greater_than_180_overnight_auto_cgmdata['Overnight Percentage'].sum()/auto_total_days if not count_greater_than_180_overnight_auto_cgmdata.empty else 0
count_greater_than_250_overnight_auto_cgmdata_percentage = count_greater_than_250_overnight_auto_cgmdata['Overnight Percentage'].sum()/auto_total_days if not count_greater_than_250_overnight_auto_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata_percentage = count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata['Overnight Percentage'].sum()/auto_total_days if not count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata.empty else 0
count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata_percentage = count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata['Overnight Percentage'].sum()/auto_total_days if not count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata.empty else 0
count_less_than_70_overnight_auto_cgmdata_percentage = count_less_than_70_overnight_auto_cgmdata['Overnight Percentage'].sum()/auto_total_days if not count_less_than_70_overnight_auto_cgmdata.empty else 0
count_less_than_54_overnight_auto_cgmdata_percentage = count_less_than_54_overnight_auto_cgmdata['Overnight Percentage'].sum()/auto_total_days if not count_less_than_54_overnight_auto_cgmdata.empty else 0

print('count_greater_than_180_overnight_manual_cgmdata_percentage: ', count_greater_than_180_overnight_manual_cgmdata_percentage)
print('count_greater_than_250_overnight_manual_cgmdata_percentage: ', count_greater_than_250_overnight_manual_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata_percentage: ', count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata_percentage: ', count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata_percentage)
print('count_less_than_70_overnight_manual_cgmdata_percentage: ', count_less_than_70_overnight_manual_cgmdata_percentage)
print('count_less_than_54_overnight_manual_cgmdata_percentage: ', count_less_than_54_overnight_manual_cgmdata_percentage)

print('count_greater_than_180_overnight_auto_cgmdata_percentage:', count_greater_than_180_overnight_auto_cgmdata_percentage)
print('count_greater_than_250_overnight_auto_cgmdata_percentage:', count_greater_than_250_overnight_auto_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata_percentage:', count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata_percentage)
print('count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata_percentage:', count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata_percentage)
print('count_less_than_70_overnight_auto_cgmdata_percentage:', count_less_than_70_overnight_auto_cgmdata_percentage)
print('count_less_than_54_overnight_auto_cgmdata_percentage:', count_less_than_54_overnight_auto_cgmdata_percentage)

count_greater_than_180_overnight_manual_cgmdata_percentage:  4.535590277777777
count_greater_than_250_overnight_manual_cgmdata_percentage:  0.9548611111111112
count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata_percentage:  17.100694444444446
count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata_percentage:  12.565104166666666
count_less_than_70_overnight_manual_cgmdata_percentage:  0.6293402777777778
count_less_than_54_overnight_manual_cgmdata_percentage:  0
count_greater_than_180_overnight_auto_cgmdata_percentage: 2.5857386716325266
count_greater_than_250_overnight_auto_cgmdata_percentage: 0.40347610180012417
count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata_percentage: 20.152467411545626
count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata_percentage: 17.582247051520792
count_less_than_70_overnight_auto_cgmdata_percentage: 0.6265518311607696
count_less_than_54_overnight_auto_cgmdata_percentage: 0.15130353817504658


In [254]:
list1_manual = [count_greater_than_180_overnight_manual_cgmdata_percentage, count_greater_than_250_overnight_manual_cgmdata_percentage, count_greater_equal_70_and_less_equal_180_overnight_manual_cgmdata_percentage, count_greater_equal_70_and_less_equal_150_overnight_manual_cgmdata_percentage, count_less_than_70_overnight_manual_cgmdata_percentage, count_less_than_54_overnight_manual_cgmdata_percentage, count_greater_than_180_daytime_manual_cgmdata_percentage, count_greater_than_250_daytime_manual_cgmdata_percentage, count_greater_equal_70_and_less_equal_180_daytime_manual_cgmdata_percentage, count_greater_equal_70_and_less_equal_150_daytime_manual_cgmdata_percentage, count_less_than_70_daytime_manual_cgmdata_percentage, count_less_than_54_daytime_manual_cgmdata_percentage, count_greater_than_180_wholeday_manual_cgmdata_percentage, count_greater_than_250_wholeday_manual_cgmdata_percentage, count_greater_equal_70_and_less_equal_180_wholeday_manual_cgmdata_percentage, count_greater_equal_70_and_less_equal_150_wholeday_manual_cgmdata_percentage, count_less_than_70_wholeday_manual_cgmdata_percentage, count_less_than_54_wholeday_manual_cgmdata_percentage]
list2_auto = [count_greater_than_180_overnight_auto_cgmdata_percentage, count_greater_than_250_overnight_auto_cgmdata_percentage, count_greater_equal_70_and_less_equal_180_overnight_auto_cgmdata_percentage, count_greater_equal_70_and_less_equal_150_overnight_auto_cgmdata_percentage, count_less_than_70_overnight_auto_cgmdata_percentage, count_less_than_54_overnight_auto_cgmdata_percentage, count_greater_than_180_daytime_auto_cgmdata_percentage, count_greater_than_250_daytime_auto_cgmdata_percentage, count_greater_equal_70_and_less_equal_180_daytime_auto_cgmdata_percentage, count_greater_equal_70_and_less_equal_150_daytime_auto_cgmdata_percentage, count_less_than_70_daytime_auto_cgmdata_percentage, count_less_than_54_daytime_auto_cgmdata_percentage, count_greater_than_180_wholeday_auto_cgmdata_percentage, count_greater_than_250_wholeday_auto_cgmdata_percentage, count_greater_equal_70_and_less_equal_180_wholeday_auto_cgmdata_percentage, count_greater_equal_70_and_less_equal_150_wholeday_auto_cgmdata_percentage, count_less_than_70_wholeday_auto_cgmdata_percentage, count_less_than_54_wholeday_auto_cgmdata_percentage]
print_df = pd.DataFrame(list1_manual).T
print_df = print_df.append(pd.Series(list2_auto), ignore_index=True)
print_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,4.535590,0.954861,17.100694,12.565104,0.629340,0.000000,22.829861,8.463542,34.830729,27.105035,4.578993,1.801215,27.365451,9.418403,51.931424,39.670139,5.208333,1.801215
1,2.585739,0.403476,20.152467,17.582247,0.626552,0.151304,19.617086,4.564323,45.361965,33.988982,3.338377,0.989292,22.202824,4.967800,65.514432,51.571229,3.964929,1.140596


In [255]:
print_df.to_csv('Results.csv', header=False, index=False)